# Python 3.10 preferred (avoid <3.8)

In [ ]:
# https://github.com/ansys/pymechanical
# pip install ansys-mechanical-core

# Import Libraries

In [ ]:
from ansys.mechanical.core import launch_mechanical, App, find_mechanical, global_variables

# pyMechanical Remote Sessions

- The Mechanical gRPC interface requires Mechanical 2023 R1 or later.
- SUPPORTED_MECHANICAL_EMBEDDING_VERSIONS_WINDOWS = ``{241: "2024R1", 232: "2023R2", 231: "2023R1"}``
- batch: Whether to launch Mechanical in batch mode. The default is ``True``
- start_instance: Whether to launch and connect to a new Mechanical instance. The default is ``None``, in which case an attempt is made to connect to an existing Mechanical instance. When ``True``, a local instance of Mechanical is launched.
- loglevel: Level of messages to print to the console. ``"WARNING"``, ``"ERROR"``, ``"INFO"``
- log_mechanical: Path to the output file on the local disk to write every script command to.
- verbose_mechanical: Whether to enable printing of all output when launching and running a Mechanical instance.
- cleanup_on_exit: Whether to exit Mechanical when Python exits. The default is ``True``

In [ ]:
mechanical = launch_mechanical(
    exec_file = None,
    batch= False, 
    start_instance = True, 
    loglevel= 'INFO', 
    log_mechanical='pymechanical_log.txt', 
    verbose_mechanical= True)
print(mechanical)

In [ ]:
# mechanical.exit(force=True)

In [ ]:
wb_exe = find_mechanical(232)[0]
print(wb_exe)

In [ ]:
proj_directory = mechanical.run_python_script("ExtAPI.DataModel.Project.ProjectDirectory")
print(proj_directory)
proj_directory = mechanical.project_directory
print(proj_directory)
# mechanical.upload(file_name="", file_location_destination=project_directory)

In [ ]:
mechanical.list_files()

## Geometry Import

In [ ]:
geometryPath = r"D:\repos\AnsysScripting\WIP\pyMechanical\geom.scdoc"
print(geometryPath)

- mechanical.run_python_script returns only string and it corrsponds to last output. If list output is expected, convert into string.

In [ ]:
script_GeometryImport = f"""
geomImport = Model.GeometryImportGroup.AddGeometryImport()
# Format of Geometry file
geomImport_format = Ansys.Mechanical.DataModel.Enums.GeometryImportPreference.Format.Automatic

#Preferences for Geometry file
geomImport_preferences = Ansys.ACT.Mechanical.Utilities.GeometryImportPreferences()

# geomImport_preferences.ProcessSolids = True
# geomImport_preferences.ProcessSurfaces = False
# geomImport_preferences.ProcessLines = False

# geomImport_preferences.ProcessNamedSelections = True
# geomImport_preferences.NamedSelectionKey = "NSel"

# geomImport_preferences.ProcessMaterialProperties = True
# geomImport_preferences.MixedImportResolution = GeometryImportPreference.MixedImportResolution.Solid

# geomImport_preferences.ProcessCoordinateSystems = True
# geomImport_preferences.CoordinateSystemKey = "CSys"
geomImport.Import(r'{geometryPath}', geomImport_format, geomImport_preferences)
"""

In [ ]:
mechanical.run_python_script(script_GeometryImport)
mechanical.log_message("INFO", "Geometry Import Success")

In [ ]:
mechanical.run_python_script("geomImport.ToString()")

## Create Connections

In [ ]:
script_Connections = """
connections = Model.Connections.AddConnectionGroup()
connections.CreateAutomaticConnections()
"""

In [ ]:
mechanical.run_python_script(script_Connections)

## Generate Mesh

In [ ]:
script_Mesh = """
mesh = Model.Mesh
mesh.GenerateMesh()
"""

In [ ]:
mechanical.run_python_script(script_Mesh)

## Create Named Selection

In [ ]:
script_NS = """
selinfo = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
ns_faceForce = Model.AddNamedSelection()
ns_faceForce.Name = "ns_faceForce"
selinfo.Ids = [35]
ns_faceForce.Location = selinfo
ns_faceFixedSupport = Model.AddNamedSelection()
ns_faceFixedSupport.Name = "ns_faceFixedSupport"
selinfo.Ids = [11]
ns_faceFixedSupport.Location = selinfo
"""

In [ ]:
mechanical.run_python_script(script_NS)

## Add Static Analysis

In [ ]:
script_Analysis = """
analysis = Model.AddStaticStructuralAnalysis()
fixedsupport = analysis.AddFixedSupport()
fixedsupport.Location = ns_faceFixedSupport
force = analysis.AddForce()
force.Location = ns_faceForce
force.Magnitude.Inputs[0].DiscreteValues = [Quantity('0 [sec]'),Quantity('1 [sec]')]
force.Magnitude.Output.DiscreteValues = [Quantity('0 [N]'),Quantity('100 [N]')]
"""

In [ ]:
mechanical.run_python_script(script_Analysis)

## Solve

In [ ]:
mechanical.run_python_script("analysis.Solve()")

## Solution & Result

In [ ]:
script_Solution = """
solution = analysis.Solution
totalDeformation = solution.AddTotalDeformation()
totalDeformation.Location = ns_faceForce
solution.EvaluateAllResults()
"""

In [ ]:
mechanical.run_python_script(script_Solution)

### Save

In [ ]:
script_Save = """ExtAPI.DataModel.Project.Save(r'D:\\repos\\AnsysScripting\\WIP\\pyMechanical\\pymech.mechdb')
"""

In [ ]:
mechanical.run_python_script(script_Save)

### Exit

In [ ]:
mechanical.exit(force=True)

In [ ]:
mechanical.run_python_script('ExtAPI.DataModel.Project.Open(r"D:\\repos\\AnsysScripting\\WIP\\pyMechanical\\pymech.mechdb")')

# PyMechanical Embedded Session

In [ ]:
app = App(version=232)
print(app)
# Add global API to Python
globals().update(global_variables(app))

In [ ]:
app.open(r"D:\repos\AnsysScripting\WIP\pyMechanical\pymech.mechdb")

In [ ]:
ExtAPI    # Application.ExtAPI
DataModel    #Application.DataModel
Model    #Application.DataModel.Project.Model
Tree    #Application.DataModel.Tree 
Graphics    #Application.ExtAPI.Graphics

In [ ]:
ExtAPI.DataModel.Project.Model

In [ ]:
Model

In [ ]:
app.save(r"D:\repos\AnsysScripting\WIP\pyMechanical\pymech.mechdb")

In [ ]:
app.exit()